In [120]:
import git
from git import Repo

In [123]:
#uncomment for activemq

path_to_folders = "activemq"
vcc_commit = "6f03921b31d9fefeddb0f4fa63150ed1f94a14b" 
#vcc_commit = "1236bbf6f455f429d3c1c5996474381a6a649d4e" 
pathName = "activemq"
repo = Repo(path_to_folders)

#uncomment for orbeon

# path_to_folders = "orbeon-forms"
# vcc_commit = "aba6681660f65af7f1676434da68c10298c30200" 
# pathName = "orbeon"
#repo = Repo(path_to_folders)

#uncomment for openjpa

# path_to_folders = "openjpa"
# vcc_commit = "87a4452be08b4f97274d0ccfac585ae85841e470" 
# pathName = "openjpa"
#repo = Repo(path_to_folders)



In [124]:
#find changed files 
show_data = repo.git.show("--stat=200", vcc_commit).splitlines()
affected_files = []
file_paths_count = 0
for line in show_data:
    if pathName in line: 
        line = line.split('|')[0]
        line = line.strip()
        affected_files.append(line)
        file_paths_count = file_paths_count+1


In [125]:
#6a print commit message

show_data = repo.git.show("-s", vcc_commit).splitlines()
for line in show_data:
    print(line)

commit 6f03921b31d9fefeddb0f4fa63150ed1f94a14b1
Author: Dejan Bosanac <dejan@nighttale.net>
Date:   Fri Oct 16 15:44:02 2015 +0200

    https://issues.apache.org/jira/browse/AMQ-6013 - restrict classes which can be serialized inside the broker


In [126]:
#6b print total files were affected in the current VCC
print(str(len(affected_files)) + " files changed")

7 files changed


In [127]:
#6c print total directories were affected in the current VCC
affected_directories = []
show_data = repo.git.show("--dirstat", vcc_commit).splitlines()
for line in show_data:
    if '%' in line:   
        line = line.split('%')[1]
        line = line.strip()
        affected_directories.append(line)
        #print(line)

print(str(len(affected_directories)) + " affected directories")

5 affected directories


In [128]:
#6d, 6e print number added and deleted lines (including comments and blank lines)

lines_changed = repo.git.show("--pretty=tformat:", "--shortstat", vcc_commit).splitlines()
for line in lines_changed:
    line = line.split()

print("Total insertions inc comments and blank: " + str(line[3]))
print("Total deletions inc comments and blank: " + str(line[5]))

Total insertions inc comments and blank: 115
Total deletions inc comments and blank: 15


In [129]:
#6i total number times each affected file of the current VCC been modified in the past since their creation 
for file in affected_files:
    modified = repo.git.log("--format=%ad", vcc_commit, "--", file).splitlines()
    print(os.path.basename(file), "modified:", len(modified), "times")

ClassLoadingAwareObjectInputStream.java modified: 3 times
XStreamWireFormat.java modified: 3 times
JmsFrameTranslator.java modified: 7 times
XStreamSupport.java modified: 1 times
StompTestSupport.java modified: 6 times
XStreamBrokerContext.java modified: 3 times
MessageQuery.java modified: 7 times


In [136]:
#6j what developers have modified each file since creation
total_author_list = []

for file in affected_files:
    author_list = repo.git.log("--format=%an", vcc_commit, "--", file).splitlines()
    total_author_list = total_author_list + author_list
    all_authors = list(set(author_list))
    print("")
    print(os.path.basename(file))
    for line in all_authors:
        print(line)



ClassLoadingAwareObjectInputStream.java
Dejan Bosanac
Claus Ibsen
Hiram R. Chirino

XStreamWireFormat.java
Dejan Bosanac
Hiram R. Chirino
Timothy Bish

JmsFrameTranslator.java
Bosanac Dejan
Dejan Bosanac
gtully
Timothy Bish
Hiram R. Chirino
Timothy A. Bish

XStreamSupport.java
Dejan Bosanac

StompTestSupport.java
Dejan Bosanac
Timothy A. Bish
Timothy Bish

XStreamBrokerContext.java
Dejan Bosanac
gtully
Timothy A. Bish

MessageQuery.java
Robert Davies
Bosanac Dejan
James Strachan
Dejan Bosanac
Hiram R. Chirino
Gary Tully
Claus Ibsen


In [138]:
#6k For each developer identified, how many commits have each of them submitted? 
commit_authors = repo.git.shortlog("-sne", vcc_commit).splitlines()

author_commits = []

for commit_author in commit_authors:
    commit, author = commit_author.split("\t")
    author_commits.append([int(commit), author])

num_file = len(affected_files)
print("avg num developers: "+ str(len(total_author_list)/num_file))
total_author_list2 = list(set(total_author_list))

print("total num developers: " + str(len(total_author_list2)))
df = pd.DataFrame(author_commits, columns=['Commit', 'Author'])

for line in total_author_list2:
    row = df[df["Author"].str.contains(line)]
    commits = row["Commit"].iloc[0]
    print(line, ":", commits, " Commits")

avg num developers: 4.285714285714286
total num developers: 10
Robert Davies : 1182  Commits
Bosanac Dejan : 942  Commits
James Strachan : 916  Commits
Dejan Bosanac : 113  Commits
gtully : 1201  Commits
Timothy Bish : 147  Commits
Hiram R. Chirino : 1408  Commits
Timothy A. Bish : 843  Commits
Gary Tully : 1201  Commits
Claus Ibsen : 137  Commits
